# Data scraping and creating dataframe

In [3]:
pip install beautifulsoup4


     |████████████████████████████████| 112kB 16.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 7.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
#define url for requests package
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(url)
html = r.text #use .text to remove html code

soup = BeautifulSoup(html)
table = soup.find('table', {'class':'wikitable'}) #select table called wikitable
rows = table.find_all('tr') #finding all the rows denoted in the html as 'tr'
data = [] #loop through each row selecting the data inbetween html 'td' parameters and generate list of tuples
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

result = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood']) 
pd.set_option('display.max_rows', None) #allow display of whole dataframe for inspection if needed
result.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
#filter table to remove rows that have no Borough assigned
has_b_val  = result[result['Borough']!='Not assigned'] 
has_b_val.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
toronto_wiki = has_b_val #all missing neighbourhood values were removed in previous filtering so no more further work required
toronto_wiki.reset_index(drop = True, inplace = True)
toronto_wiki.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
print('original table shape:',result.shape)
print('updated table shape:', toronto_wiki.shape)


original table shape: (180, 3)
updated table shape: (103, 3)


# Combining with Geolocation data

In [9]:
geo_coord_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_coord_df.head(15)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
geo_coord_df.columns = ['PostalCode', 'Latitude', 'Longitude']
df_toronto_latlon = pd.merge(toronto_wiki, geo_coord_df, on='PostalCode')
df_toronto_latlon.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
